In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import torch
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizer
from tqdm.auto import tqdm
import torch.nn.functional as F

from transformers import BertForSequenceClassification, BertTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import tempfile
from huggingface_hub import HfFolder, Repository
import os

from datasets import load_dataset


In [4]:
base_path = '/content/drive/Shareddrives/World Bank/NLP Project/'

# Re-training BERT

We re-train BERT making sure to not train it on any real data, only synthetic.

In [5]:
file_name = 'Data/train_final_multiple_sources.csv'
df = pd.read_csv(base_path + file_name)

Prompt that we use to generate the data:

```python
These are example sentences that mention a certain data source and its subsequent usage for research purposes:
"Using citation data from the Google Scholar platform , this study revealed the heterogen eous quality of the research products selected."
"In another study, a survey of senior World Bank operational staff was used to assess their awareness about research products, such as those produced by DEC (Ravallion and Wagstaff 2011)."

In order to train a model to detect dataset mentions and usage, I want you to generate similar sentences to these that mention a dataset and its use for research in the following format:
{"text": "We examined patterns of internet usage using data from the Pew Research Center's Internet & Technology."}
A few things to note:
1. You must not use the words "dataset" or "database"
2. You must not make the research context too obvious, in order to not give the model too many "clues": please avoid vocabulary such as "study", "research", etc.
```





In [ ]:
df_gpt_negative = load_dataset("notrichardren/gpt_generated_10k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/10869 [00:00<?, ? examples/s]

In [ ]:
gpt_data = pd.DataFrame(df_gpt_negative ['train'])
gpt_data = gpt_data[['claim']]
gpt_data['data_origin'] = 'https://huggingface.co/datasets/notrichardren/gpt_generated_10k'
gpt_data['label'] = 0
gpt_data.rename(columns = {"claim":"text"}, inplace=True)

<ipython-input-8-a7400e2c0b3c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpt_data['data_origin'] = 'https://huggingface.co/datasets/notrichardren/gpt_generated_10k'
<ipython-input-8-a7400e2c0b3c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpt_data['label'] = 0
<ipython-input-8-a7400e2c0b3c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

In [6]:
# Split data into real and synthetic

df_real = df.query("data_origin.str.startswith('Coleridge')")
df_real.dropna(inplace=True)
df_synthetic = df.query("data_origin.str.startswith('Coleridge') == False")

df_real_positive = df_real.query("label == 1")
df_real_negative = df_real.query("label == 0")
# df_synthetic_positive = df_synthetic.query("label == 1")
# df_synthetic_negative = df_synthetic.query("label == 0")
# #df_synthetic_negative = pd.concat([df_synthetic_negative, gpt_data.head(1944)])
# df_synthetic_negative = gpt_data

print("real", df_real.shape)
print("synthetic", df_synthetic.shape)
print('real positive', df_real_positive.shape)
print('real negative', df_real_negative.shape)
# print("synthetic positive", df_synthetic_positive.shape)
# print("synthetic negative", df_synthetic_negative.shape)



# # Balance the synthetic dataset
# min_count_synthetic = min(len(df_synthetic_positive), len(df_synthetic_negative))

# df_synthetic_positive_balanced = df_synthetic_positive.sample(n=min_count_synthetic, random_state=1)
# df_synthetic_negative_balanced = df_synthetic_negative.sample(n=min_count_synthetic, random_state=1)
# #print(df_synthetic_negative_balanced.isna().sum())

# # Create the balanced synthetic dataset
# df_synthetic_balanced = pd.concat([df_synthetic_positive_balanced, df_synthetic_negative_balanced], ignore_index=True)

# # Calculate the required training size from the synthetic dataset
# synthetic_training_size = int(0.8 * len(df_synthetic_balanced))

# # Train using synthetic data only
# df_train_synthetic = df_synthetic_balanced.sample(n=synthetic_training_size, random_state=1)

# # The remainder of the synthetic data will be used as part of the validation set
# df_val_synthetic_remaining = df_synthetic_balanced.drop(df_train_synthetic.index)

# # For the real dataset, we use as much as possible without exceeding the validation size balance
# # Initially, assume we will use all real data and check if it fits
# df_real_balanced = pd.concat([df_real_positive, df_real_negative])

# # Determine how many real data can be added without exceeding the total expected validation size
# expected_validation_size = len(df_synthetic_balanced) - synthetic_training_size

# # If there's enough room, add all real data to validation; otherwise, sample appropriately
# if len(df_real_balanced) <= expected_validation_size - len(df_val_synthetic_remaining):
#     df_val = pd.concat([df_val_synthetic_remaining, df_real_balanced])
# else:
#     additional_real_data_needed = expected_validation_size - len(df_val_synthetic_remaining)
#     # Ensuring the balance within the real data added to validation
#     balanced_real_to_add = df_real_balanced.sample(n=additional_real_data_needed, random_state=1)
#     df_val = pd.concat([df_val_synthetic_remaining, balanced_real_to_add])

# # Shuffle the validation dataset
# df_val = df_val.sample(frac=1, random_state=1)

# # Print the shapes to verify
# print("Training data shape (synthetic only):", df_train_synthetic.shape)
# print("Validation data shape (mixed real and synthetic):", df_val.shape)


real (6916, 4)
synthetic (8353, 4)
real positive (254, 4)
real negative (6662, 4)


<ipython-input-6-450c402bb48b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_real.dropna(inplace=True)


In [7]:
from transformers import BertForSequenceClassification, BertTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import tempfile
from huggingface_hub import HfFolder, Repository
import os
from sklearn.metrics import roc_auc_score
import copy

# FINE-TUNE BERT


class TrainBERTClassifier:
    def __init__(
        self,
        model_name="bert-base-uncased",
        num_labels=2,
        max_length=32,
        batch_size=16,
        learning_rate=5e-7,
        epsilon=1e-8,
        epochs=2000,
        patience=3,
        tolerance=1e-4,
        optimizer_type='AdamW'
    ):

        self.optimizer_type = optimizer_type
        self.patience = patience
        self.tolerance = tolerance
        self.best_val_roc_auc = 0.0  # Initialize the best ROC-AUC score
        self.epochs_no_improve = 0  # Initialize the count for epochs without improvement
        self.best_model_state = None

        """
        Initializes the BERT Classifier for training and evaluation.

        Args:
        model_name (str): Name or path of the pre-trained model.
        num_labels (int): Number of labels for classification.
        max_length (int): Maximum length of the input sequence.
        batch_size (int): Batch size for training and evaluation.
        learning_rate (float): Learning rate for the optimizer.
        epsilon (float): Epsilon value for the optimizer.
        epochs (int): Number of training epochs.
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = BertForSequenceClassification.from_pretrained(
            model_name, num_labels=num_labels
        ).to(self.device)
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.max_length = max_length
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.epochs = epochs

    def preprocess(self, texts):
        """
        Preprocesses a list of texts for BERT model.

        Args:
        texts (list of str): List of texts to be preprocessed.

        Returns:
        TensorDataset: A dataset of input IDs and attention masks.
        """
        input_ids = []
        attention_masks = []

        for text in texts:
            encoding = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_length,
                padding="max_length",
                truncation=True,
                return_attention_mask=True,
                return_tensors="pt",
            )
            input_ids.append(encoding["input_ids"])
            attention_masks.append(encoding["attention_mask"])
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        return TensorDataset(input_ids, attention_masks)



    def select_optimizer(self, parameters):
      """
      Selects the optimizer based on the specified optimizer type.

      Args:
      parameters: Model parameters to optimize.

      Returns:
      Optimizer: The selected optimizer.
      """
      if self.optimizer_type == 'AdamW':
          return torch.optim.AdamW(parameters, lr=self.learning_rate, eps=self.epsilon)
      elif self.optimizer_type == 'SGD':
          return torch.optim.SGD(parameters, lr=self.learning_rate)
      elif self.optimizer_type == 'RMSprop':
          return torch.optim.RMSprop(parameters, lr=self.learning_rate, eps=self.epsilon)
      elif self.optimizer_type == 'Adam':
          return torch.optim.Adam(parameters, lr=self.learning_rate, eps=self.epsilon)
      elif self.optimizer_type == 'Adagrad':
          return torch.optim.Adagrad(parameters, lr=self.learning_rate)
      elif self.optimizer_type == 'Adadelta':
          return torch.optim.Adadelta(parameters, lr=self.learning_rate)
      else:
          raise ValueError(f"Unsupported optimizer type: {self.optimizer_type}")



    @staticmethod
    def flat_accuracy(preds, labels):
        """
        Calculates the accuracy of the predictions based on the comparison with true labels.

        Args:
        preds: Numpy array of predictions.
        labels: Numpy array of actual labels.

        Returns:
        float: Accuracy score.
        """
        pred_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        return np.sum(pred_flat == labels_flat) / len(labels_flat)

    def b_tp(self, preds, labels):
        """
        Calculates the number of true positives.

        Args:
        preds (list or array): Predicted labels.
        labels (list or array): Actual labels.

        Returns:
        int: Number of true positives.
        """
        return sum((pred == 1) and (label == 1) for pred, label in zip(preds, labels))

    def b_fp(self, preds, labels):
        """
        Calculates the number of false positives.

        Args:
        preds (list or array): Predicted labels.
        labels (list or array): Actual labels.

        Returns:
        int: Number of false positives.
        """
        return sum((pred == 1) and (label == 0) for pred, label in zip(preds, labels))

    def b_tn(self, preds, labels):
        """
        Calculates the number of true negatives.

        Args:
        preds (list or array): Predicted labels.
        labels (list or array): Actual labels.

        Returns:
        int: Number of true negatives.
        """
        return sum((pred == 0) and (label == 0) for pred, label in zip(preds, labels))

    def b_fn(self, preds, labels):
        """
        Calculates the number of false negatives.

        Args:
        preds (list or array): Predicted labels.
        labels (list or array): Actual labels.

        Returns:
        int: Number of false negatives.
        """
        return sum((pred == 0) and (label == 1) for pred, label in zip(preds, labels))

    def b_metrics(self, preds, labels):
        """
        Calculates binary classification metrics including accuracy, precision, recall, and specificity.

        Args:
        preds (array): Model's probability predictions before applying threshold.
        labels (array): Actual labels.

        Returns:
        tuple: A tuple containing the accuracy, precision, recall, and specificity.
        """
        preds = np.argmax(preds, axis=1).flatten()
        labels = labels.flatten()

        tp = self.b_tp(preds, labels)
        tn = self.b_tn(preds, labels)
        fp = self.b_fp(preds, labels)
        fn = self.b_fn(preds, labels)

        b_accuracy = (tp + tn) / len(labels)
        b_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        b_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

        return b_accuracy, b_precision, b_recall, b_specificity

    def compute_roc_auc(self, preds, labels):
        """
        Computes the ROC-AUC score.

        Args:
        preds (np.array): Raw model predictions (logits or probabilities).
        labels (np.array): Actual labels.

        Returns:
        float: ROC-AUC score.
        """
        # Convert softmax output to positive class probabilities for ROC-AUC calculation
        preds_proba = np.exp(preds)[:, 1] / np.sum(np.exp(preds), axis=1)
        roc_auc = roc_auc_score(labels, preds_proba)
        return roc_auc

    def train(self, train_texts, train_labels, val_texts, val_labels):
        """
        Trains and evaluates the BERT model.

        Args:
        train_texts (list of str): Training texts.
        train_labels (list of int): Training labels.
        val_texts (list of str): Validation texts.
        val_labels (list of int): Validation labels.
        """
        train_dataset = self.preprocess(train_texts)
        val_dataset = self.preprocess(val_texts)

        train_labels = torch.tensor(train_labels)
        val_labels = torch.tensor(val_labels)

        train_dataset = TensorDataset(
            train_dataset.tensors[0], train_dataset.tensors[1], train_labels
        )
        val_dataset = TensorDataset(
            val_dataset.tensors[0], val_dataset.tensors[1], val_labels
        )

        train_dataloader = DataLoader(
            train_dataset,
            sampler=RandomSampler(train_dataset),
            batch_size=self.batch_size,
        )
        validation_dataloader = DataLoader(
            val_dataset,
            sampler=SequentialSampler(val_dataset),
            batch_size=self.batch_size,
        )

        # Initialize the optimizer
        optimizer = self.select_optimizer(self.model.parameters())


        for epoch in range(self.epochs):
            self.model.train()
            total_loss = 0

            for batch in train_dataloader:
                batch = tuple(b.to(self.device) for b in batch)
                b_input_ids, b_input_mask, b_labels = batch
                self.model.zero_grad()
                outputs = self.model(
                    b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels,
                )
                loss = outputs.loss
                total_loss += loss.item()
                loss.backward()
                optimizer.step()
            avg_train_loss = total_loss / len(train_dataloader)
            print(f"Epoch {epoch + 1}")
            print(f"Training loss: {avg_train_loss}")

            self.model.eval()
            eval_accuracy = 0
            eval_metrics = np.zeros(
                4
            )  # To store accuracy, precision, recall, specificity
            nb_eval_steps = 0

            logits_all = []
            label_ids_all = []

            for batch in validation_dataloader:
                batch = tuple(b.to(self.device) for b in batch)
                b_input_ids, b_input_mask, b_labels = batch

                with torch.no_grad():
                    outputs = self.model(
                        b_input_ids, token_type_ids=None, attention_mask=b_input_mask
                    )
                logits = outputs.logits.detach().cpu().numpy()
                label_ids = b_labels.to("cpu").numpy()

                logits_all.append(logits)
                label_ids_all.append(label_ids)

                tmp_eval_accuracy = self.flat_accuracy(logits, label_ids)
                eval_accuracy += tmp_eval_accuracy
                metrics = self.b_metrics(logits, label_ids)
                eval_metrics += np.array(metrics)
                nb_eval_steps += 1

            # Concatenate all logits and labels from the validation loop
            logits_all = np.concatenate(logits_all, axis=0)
            label_ids_all = np.concatenate(label_ids_all, axis=0)

            # Compute ROC-AUC after processing all validation batches
            roc_auc = self.compute_roc_auc(logits_all, label_ids_all)
            print(f"Validation ROC-AUC: {roc_auc}")
            print(f"Validation Accuracy: {eval_accuracy / nb_eval_steps}")
            print(f"Validation Precision: {eval_metrics[1] / nb_eval_steps}")
            print(f"Validation Recall: {eval_metrics[2] / nb_eval_steps}")
            print(f"Validation Specificity: {eval_metrics[3] / nb_eval_steps}")


            if roc_auc > self.best_val_roc_auc + self.tolerance:
                self.best_val_roc_auc = roc_auc
                self.epochs_no_improve = 0
                self.best_model_state = copy.deepcopy(self.model.state_dict())  # Save the best model state
            else:
                self.epochs_no_improve += 1
                print(f"No improvement in validation ROC-AUC for {self.epochs_no_improve} consecutive epochs.")

                if self.epochs_no_improve >= self.patience:
                    print("Early stopping triggered.")
                    if self.best_model_state is not None:
                        print("Restoring best model weights!")
                        self.model.load_state_dict(self.best_model_state)  # Restore the best model state
                    break


    def save_model_and_tokenizer(self, save_directory):
        """
        Saves the trained model and tokenizer to the specified directory.

        Args:
        save_directory (str): The directory to save the model and tokenizer.
        """
        # Ensure the save directory exists
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)

        # Save the model
        model_save_path = os.path.join(save_directory, "model")
        self.model.save_pretrained(model_save_path)

        # Save the tokenizer
        tokenizer_save_path = os.path.join(save_directory, "tokenizer")
        self.tokenizer.save_pretrained(tokenizer_save_path)

        print(f"Model and tokenizer saved in {save_directory}")

    def push_to_huggingface_hub(self, commit_message="Push model to Hugging Face Hub"):
        """
        Pushes the fine-tuned model to the Hugging Face Hub, prompting the user for their username and access token.

        Args:
        commit_message (str): Commit message for the repository.
        """
        # Prompt the user for their Hugging Face username and access token

        hf_username = input("Enter your Hugging Face username: ")
        hf_access_token = input("Enter your Hugging Face access token: ")

        # Create a temporary directory to store model files

        temp_dir = tempfile.mkdtemp()

        # Save the model to the temporary directory

        self.model.save_pretrained(temp_dir)

        # Define the desired model name on the Hugging Face Hub

        model_name_on_hub = input("Enter your desired model name on Hugging Face Hub: ")

        # Initialize a repository in the temporary directory, set to push to the Hugging Face Hub

        repo = Repository(
            local_dir=temp_dir,
            clone_from=f"{hf_username}/{model_name_on_hub}",
            use_auth_token=hf_access_token,
        )

        # Push the model to the Hugging Face Hub

        repo.push_to_hub(commit_message=commit_message)


# LOAD BERT AND USE IT FOR CLASSIFICATION PURPOSES


class BERTClassifier:
    def __init__(
        self, model_path="jamesliounis/DataBERT", tokenizer_name="bert-base-uncased"
    ):
        """
        Initializes the BERT Classifier with the fine-tuned model and tokenizer.

        Args:
        model_path (str): Path to the fine-tuned model directory.
        In this case, points to HuggingFace where we have deployed the fine-tuned model.
        tokenizer_name (str): Pretrained tokenizer name or path.
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = BertForSequenceClassification.from_pretrained(model_path).to(
            self.device
        )
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_name)

    def preprocess(self, input_text):
        """
        Preprocesses the input text for BERT model.

        Args:
        input_text (str): Text to be preprocessed.

        Returns:
        torch.Tensor: Tensor of token IDs.
        torch.Tensor: Tensor of attention masks.
        """
        encoding = self.tokenizer.encode_plus(
            input_text,
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=32,  # Pad & truncate all sentences.
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",  # Return PyTorch tensors
        )

        input_ids = encoding["input_ids"].to(self.device)
        attention_mask = encoding["attention_mask"].to(self.device)

        return input_ids, attention_mask

    def predict(self, input_text):
      """
      Generates predictions and confidence scores for the input text.

      Args:
      input_text (str): Text to generate prediction for.

      Returns:
      str: Predicted class ('Contains dataset mention' or 'Does not contain dataset mention').
      float: Confidence score for the prediction.
      """
      self.model.eval()

      input_ids, attention_mask = self.preprocess(input_text)

      with torch.no_grad():
          outputs = self.model(input_ids, attention_mask=attention_mask)
      logits = outputs.logits
      probabilities = F.softmax(logits, dim=1)
      predicted_class_id = logits.argmax().item()
      confidence = probabilities[0, predicted_class_id].item()

      return (
          # Binary classification
          1
          if predicted_class_id == 1
          else 0,
          # Confidence score
          confidence
      )




In [ ]:
# # Extracting training texts and labels from the synthetic training dataset
# train_data = df_synthetic
# val_data = pd.concat([df_real_positive, df_real_negative.head(1417)], ignore_index = True)

# train_texts = train_data['text'].tolist()
# train_labels = train_data['label'].tolist()

# # For the validation dataset, you combine synthetic and real data and then extract texts and labels
# # Assuming the final validation dataframe 'df_val' is properly structured with 'text' and 'label' columns
# val_texts = val_data['text'].tolist()
# val_labels = val_data['label'].tolist()

# # Now you can initialize and train your BERT classifier
# bert_classifier = TrainBERTClassifier()

# # Training the model
# bert_classifier.train(train_texts, train_labels, val_texts, val_labels)

# model_name = 'DataBERT_v3'
# model_save_path = base_path + "Trained Models/" + model_name

# bert_classifier.save_model_and_tokenizer(model_save_path)


In [ ]:
import itertools
import pandas as pd

train_data = df_synthetic
val_data = pd.concat([df_real_positive, df_real_negative.head(1417)], ignore_index=True)

train_texts = train_data['text'].tolist()
train_labels = train_data['label'].tolist()
val_texts = val_data['text'].tolist()
val_labels = val_data['label'].tolist()

# Extend the hyperparameter grid to include optimizer types
param_grid = {
    'learning_rate': [5e-7, 5e-8, 5e-5, 3e-5, 2e-5],
    'batch_size': [16, 32],
    'epochs': [2000],
    'optimizer_type': ['AdamW', 'SGD', 'RMSprop', 'Adam']
}

# Store the best model and hyperparameters
best_roc_auc = 0.0
best_model = None
best_hyperparams = {}

# Iterate over all combinations of hyperparameters
for params in itertools.product(*param_grid.values()):
    hyperparams = dict(zip(param_grid.keys(), params))
    print(f"Training with parameters: {hyperparams}")

    # Initialize the BERT classifier with the current set of hyperparameters including the optimizer type
    bert_classifier = TrainBERTClassifier(
        batch_size=hyperparams['batch_size'],
        learning_rate=hyperparams['learning_rate'],
        epochs=hyperparams['epochs'],
        optimizer_type=hyperparams['optimizer_type']  # Pass the optimizer type
    )

    # Train the model
    bert_classifier.train(train_texts, train_labels, val_texts, val_labels)

    # Update the best model based on validation ROC-AUC
    if bert_classifier.best_val_roc_auc > best_roc_auc:
        best_roc_auc = bert_classifier.best_val_roc_auc
        best_model = bert_classifier
        best_hyperparams = hyperparams

# Output the best ROC-AUC score and hyperparameters
print(f"Best ROC-AUC: {best_roc_auc}")
print(f"Best Hyperparameters: {best_hyperparams}")

# Save the best model
model_name = 'DataBERT_v4'
model_save_path = base_path + "Trained Models/" + model_name
best_model.save_model_and_tokenizer(model_save_path)


Training with parameters: {'learning_rate': 5e-07, 'batch_size': 16, 'epochs': 2000, 'optimizer_type': 'AdamW'}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch 1
Training loss: 0.6392894990129635
Validation ROC-AUC: 0.5514811707111064
Validation Accuracy: 0.3025510204081633
Validation Precision: 0.15091575091575093
Validation Recall: 0.12712585034013607
Validation Specificity: 0.17636054421768707
Epoch 2
Training loss: 0.6337763835777516
Validation ROC-AUC: 0.5510755227579616
Validation Accuracy: 0.29183673469387755
Validation Precision: 0.15091575091575093
Validation Recall: 0.13069727891156463
Validation Specificity: 0.16207482993197278
No improvement in validation ROC-AUC for 1 consecutive epochs.
Epoch 3
Training loss: 0.6305723344162819
Validation ROC-AUC: 0.5508782556026651
Validation Accuracy: 0.2852891156462585
Validation Precision: 0.15091575091575093
Validation Recall: 0.13188775510204082
Validation Specificity: 0.15433673469387754
No improvement in validation ROC-AUC for 2 consecutive epochs.
Epoch 4
Training loss: 0.6295988336347941
Validation ROC-AUC: 0.5503142382431554
Validation Accuracy: 0.269812925170068
Validation Prec

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1
Training loss: 0.20046738351394297
Validation ROC-AUC: 0.5011419267722091
Validation Accuracy: 0.1488095238095238
Validation Precision: 0.15119047619047618
Validation Recall: 0.14821428571428572
Validation Specificity: 0.0017857142857142857
Epoch 2
Training loss: 0.12061210187511735
Validation ROC-AUC: 0.5371806911574303
Validation Accuracy: 0.15
Validation Precision: 0.15119047619047618
Validation Recall: 0.1494047619047619
Validation Specificity: 0.0017857142857142857
Epoch 3
Training loss: 0.08786945187452874
Validation ROC-AUC: 0.5538317061108362
Validation Accuracy: 0.15119047619047618
Validation Precision: 0.15119047619047618
Validation Recall: 0.1505952380952381
Validation Specificity: 0.0017857142857142857
Epoch 4
Training loss: 0.06698795217116066
Validation ROC-AUC: 0.5635172455948299
Validation Accuracy: 0.15178571428571427
Validation Precision: 0.15119047619047618
Validation Recall: 0.15119047619047618
Validation Specificity: 0.0017857142857142857
Epoch 5
Training l

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1
Training loss: 0.5182023798302529
Validation ROC-AUC: 0.5275729471713002
Validation Accuracy: 0.15595238095238095
Validation Precision: 0.15119047619047618
Validation Recall: 0.14821428571428572
Validation Specificity: 0.008928571428571428
Epoch 2
Training loss: 0.3280913232047517
Validation ROC-AUC: 0.5627976372395935
Validation Accuracy: 0.1523809523809524
Validation Precision: 0.15119047619047618
Validation Recall: 0.1488095238095238
Validation Specificity: 0.004761904761904762
Epoch 3
Training loss: 0.23242842760756185
Validation ROC-AUC: 0.5796959307397797
Validation Accuracy: 0.15119047619047618
Validation Precision: 0.15119047619047618
Validation Recall: 0.1505952380952381
Validation Specificity: 0.0017857142857142857
Epoch 4
Training loss: 0.17839914150037456
Validation ROC-AUC: 0.5887729982940559
Validation Accuracy: 0.1505952380952381
Validation Precision: 0.15119047619047618
Validation Recall: 0.15178571428571427
Validation Specificity: 0.0
Epoch 5
Training loss: 0.1

In [ ]:
# ============= Train BERT with best params as previously changing optimizer =====================




# Using BERT for classification purposes

In [ ]:
class BERTClassifier:
    def __init__(
        # self, model_path="jamesliounis/DataBERT", tokenizer_name="bert-base-uncased"
        self, model_path=model_save_path, tokenizer_name="bert-base-uncased"
    ):
        """
        Initializes the BERT Classifier with the fine-tuned model and tokenizer.

        Args:
        model_path (str): Path to the fine-tuned model directory.
        In this case, points to HuggingFace where we have deployed the fine-tuned model.
        tokenizer_name (str): Pretrained tokenizer name or path.
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = BertForSequenceClassification.from_pretrained(model_path).to(
            self.device
        )
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_name)

    def preprocess(self, input_text):
        """
        Preprocesses the input text for BERT model.

        Args:
        input_text (str): Text to be preprocessed.

        Returns:
        torch.Tensor: Tensor of token IDs.
        torch.Tensor: Tensor of attention masks.
        """
        encoding = self.tokenizer.encode_plus(
            input_text,
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=32,  # Pad & truncate all sentences.
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",  # Return PyTorch tensors
        )

        input_ids = encoding["input_ids"].to(self.device)
        attention_mask = encoding["attention_mask"].to(self.device)

        return input_ids, attention_mask

    def predict(self, input_text):
      """
      Generates predictions and confidence scores for the input text.

      Args:
      input_text (str): Text to generate prediction for.

      Returns:
      str: Predicted class ('Contains dataset mention' or 'Does not contain dataset mention').
      float: Confidence score for the prediction.
      """
      self.model.eval()

      input_ids, attention_mask = self.preprocess(input_text)

      with torch.no_grad():
          outputs = self.model(input_ids, attention_mask=attention_mask)
      logits = outputs.logits
      probabilities = F.softmax(logits, dim=1)
      predicted_class_id = logits.argmax().item()
      confidence = probabilities[0, predicted_class_id].item()

      return (
          "Contains dataset mention"
          if predicted_class_id == 1
          else "Does not contain dataset mention",
          confidence
      )

In [ ]:
from transformers import BertForSequenceClassification

base_path = '/content/drive/Shareddrives/World Bank/NLP Project/'
model_name = 'DataBERT_v2'
model_save_path = base_path + "Trained Models/" + model_name +"/model"

# Load the model back
model = BertForSequenceClassification.from_pretrained(model_save_path)

In [ ]:
path = base_path + '/Data/Annotated Data/pwrp_annotated_50.xlsx'

data= pd.read_excel(path)



In [ ]:
def dataset_classifier(input_sentence):
  bert_classifier = BERTClassifier()
  prediction = bert_classifier.predict(input_sentence)
  return prediction

dataset_classifier("I used data from the CIDC")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

('Contains dataset mention', 0.9996578693389893)

In [ ]:
tqdm.pandas()

data['DataBERT_v2_classification'] = data['sentence'].progress_apply(dataset_classifier)

  0%|          | 0/4896 [00:00<?, ?it/s]

In [ ]:
data = data[['sentence', 'has_data', 'confidence_score', 'has_data_v2', 'confidence_score_v2']]

In [ ]:
data.head()

,sentence,id,last_modified_date,authors,count,count_key,docna,docty,docty_key,owner,...,subsc,theme,prdln,prdln_key,projectid,trustfund,trustfund_key,has_data,DataBERT_classification,DataBERT_v2_classification
0,['Policy Research Working Paper 10704\nWhat Li...,34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Does not contain dataset mention,"('Does not contain dataset mention', 0.9092611...","(Does not contain dataset mention, 0.999603688..."
1,An objective of the series is to get the findi...,34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Does not contain dataset mention,"('Does not contain dataset mention', 0.9985073...","(Does not contain dataset mention, 0.999659061..."
2,The papers carry the \nnames of the authors an...,34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains dataset mention,"('Contains dataset mention', 0.7614002823829651)","(Does not contain dataset mention, 0.999688625..."
3,"The findings, interpretations, and conclusions...",34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Does not contain dataset mention,"('Does not contain dataset mention', 0.5594068...","(Does not contain dataset mention, 0.998246669..."
4,They do not necessarily represent the views of...,34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Does not contain dataset mention,"('Does not contain dataset mention', 0.9964689...","(Does not contain dataset mention, 0.999560177..."


In [ ]:
path = base_path + '/Data/Annotated Data/pwrp_annotated_50_latest.xlsx'

data.to_excel(path, index=False)

In [ ]:

dataset_classifier("We used data from ADNI.")

('Contains dataset mention', 0.9735932350158691)

In [ ]:
df = pd.read_excel(base_path + '/Data/Annotated Data/pwrp_annotated_50_latest.xlsx')
df.head()

,sentence,id,last_modified_date,authors,count,count_key,docna,docty,docty_key,owner,...,subsc,theme,prdln,prdln_key,projectid,trustfund,trustfund_key,has_data,DataBERT_classification,DataBERT_v2_classification
0,['Policy Research Working Paper 10704\nWhat Li...,34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Does not contain dataset mention,"('Does not contain dataset mention', 0.9092611...","('Does not contain dataset mention', 0.9996036..."
1,An objective of the series is to get the findi...,34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Does not contain dataset mention,"('Does not contain dataset mention', 0.9985073...","('Does not contain dataset mention', 0.9996590..."
2,The papers carry the \nnames of the authors an...,34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains dataset mention,"('Contains dataset mention', 0.7614002823829651)","('Does not contain dataset mention', 0.9996886..."
3,"The findings, interpretations, and conclusions...",34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Does not contain dataset mention,"('Does not contain dataset mention', 0.5594068...","('Does not contain dataset mention', 0.9982466..."
4,They do not necessarily represent the views of...,34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Does not contain dataset mention,"('Does not contain dataset mention', 0.9964689...","('Does not contain dataset mention', 0.9995601..."


In [ ]:
df.columns

Index(['sentence', 'id', 'last_modified_date', 'authors', 'count', 'count_key',
       'docna', 'docty', 'docty_key', 'owner', 'seccl', 'lang', 'lang_key',
       'entityids', 'repnb', 'docdt', 'datestored', 'volnb', 'majdocty',
       'majdocty_key', 'abstracts', 'colti', 'display_title',
       'disclosure_date', 'disclosure_type', 'pdfurl', 'txturl',
       'listing_relative_url', 'url_friendly_title', 'new_url', 'ext_pub_date',
       'disclstat', 'topicv3', 'docm_id', 'chronical_docm_id',
       'publishtoextweb_dt', 'dois', 'versiontyp', 'versiontyp_key', 'guid',
       'origu', 'no_of_pages', 'author_keys', 'available_in',
       'fullavailablein', 'url', 'admreg', 'admreg_key', 'projn', 'subsc',
       'theme', 'prdln', 'prdln_key', 'projectid', 'trustfund',
       'trustfund_key', 'has_data', 'DataBERT_classification',
       'DataBERT_v2_classification'],
      dtype='object')

In [ ]:
import ast

# Assuming df is already defined and contains the necessary columns
df = df[['sentence', 'DataBERT_classification', 'DataBERT_v2_classification']]

# Convert the string representations to actual tuples
make_tuple = lambda s: ast.literal_eval(s)
#df['DataBERT_classification'] = df['DataBERT_classification'].apply(make_tuple)
#df['DataBERT_v2_classification'] = df['DataBERT_v2_classification'].apply(make_tuple)

# Extracting class and confidence without modifications to class
df['DataBERT_class'] = df['DataBERT_classification'].apply(lambda x: x[0])
df['DataBERT_confidence'] = df['DataBERT_classification'].apply(lambda x: round(x[1], 2))

df['DataBERT_v2_class'] = df['DataBERT_v2_classification'].apply(lambda x: x[0])
df['DataBERT_v2_confidence'] = df['DataBERT_v2_classification'].apply(lambda x: round(x[1], 2))


<ipython-input-41-1c76c193f7bf>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DataBERT_class'] = df['DataBERT_classification'].apply(lambda x: x[0])
<ipython-input-41-1c76c193f7bf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DataBERT_confidence'] = df['DataBERT_classification'].apply(lambda x: round(x[1], 2))
<ipython-input-41-1c76c193f7bf>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
v1 = df[['sentence', 'DataBERT_class', 'DataBERT_confidence']]
v2 = df[['sentence', 'DataBERT_v2_class', 'DataBERT_v2_confidence']]

In [ ]:
v1['DataBERT_confidence_rank'] = v1['DataBERT_confidence'].rank(method='dense', ascending=False)
v2['DataBERT_confidence_rank'] = v2['DataBERT_v2_confidence'].rank(method='dense', ascending=False)

v1.sort_values(by='DataBERT_confidence_rank', inplace=True)
v2.sort_values(by='DataBERT_confidence_rank', inplace=True)

<ipython-input-48-d9e41078a759>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  v1['DataBERT_confidence_rank'] = v1['DataBERT_confidence'].rank(method='dense', ascending=False)
<ipython-input-48-d9e41078a759>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  v2['DataBERT_confidence_rank'] = v2['DataBERT_v2_confidence'].rank(method='dense', ascending=False)
<ipython-input-48-d9e41078a759>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

In [ ]:
v1

,sentence,DataBERT_class,DataBERT_confidence,DataBERT_confidence_rank
2447,"Second, the benefits of connectivity to young ...",Does not contain dataset mention,1.0,1.0
2557,"Tracking the evolution of impact , and cumulat...",Does not contain dataset mention,1.0,1.0
2560,∅𝑖𝑖 is a vector of \ndistrict and state fixed ...,Does not contain dataset mention,1.0,1.0
2562,"T he sample comprises 1,188 observations.",Does not contain dataset mention,1.0,1.0
2563,"For \nparsimony, we drop the lagged dependent ...",Does not contain dataset mention,1.0,1.0
...,...,...,...,...
2053,"\nWoldeyohanes TR, Woldehaimanot TE, Kerie MW ...",Does not contain dataset mention,0.5,51.0
4512,World Bank Policy Research Working Paper No.,Contains dataset mention,0.5,51.0
4130,"For example, in Madagascar, in a context of cu...",Contains dataset mention,0.5,51.0
3328,"\n \nMoro, Andrea, Tomasz Piotr Wisniewski, an...",Does not contain dataset mention,0.5,51.0


In [ ]:
v2

,sentence,DataBERT_v2_class,DataBERT_v2_confidence,DataBERT_confidence_rank
0,['Policy Research Working Paper 10704\nWhat Li...,Does not contain dataset mention,1.00,1.0
3249,“The Employment Effects of Credit Market Disru...,Does not contain dataset mention,1.00,1.0
3248,"Chodorow- Reich, G. (2014).",Does not contain dataset mention,1.00,1.0
3247,"“Gender, small firm ownership, and credit \nac...",Does not contain dataset mention,1.00,1.0
3246,"\n Chaudhuri, K., S. Sasidharan, and R. S. N. ...",Does not contain dataset mention,1.00,1.0
...,...,...,...,...
3010,"For this, we draw on the broader literature on...",Does not contain dataset mention,0.54,38.0
3685,"Nil-filers\nNext, we examined the prevalence o...",Does not contain dataset mention,0.53,39.0
2135,"In Kenya, for example, admissions data are com...",Contains dataset mention,0.52,40.0
1270,Nonfarm Microenterprise Performance and the \n...,Does not contain dataset mention,0.51,41.0


In [ ]:
base_path

'/content/drive/Shareddrives/World Bank/NLP Project/'

In [ ]:
v1.to_excel(base_path + 'databert_v1_ranked_pwrp.xlsx', index=False)
v2.to_excel(base_path + 'databert_v2_ranked_pwrp.xlsx', index=False)

In [ ]:
v1 = pd.read_excel(base_path+"Data/Annotated Data/databert_v1_ranked_pwrp.xlsx")
v1

,sentence,DataBERT_class,DataBERT_confidence,DataBERT_confidence_rank,Comments,Unnamed: 5
0,"Second, the benefits of connectivity to young ...",Does not contain dataset mention,1.0,1,NaN,NaN
1,"Tracking the evolution of impact , and cumulat...",Does not contain dataset mention,1.0,1,NaN,NaN
2,∅𝑖𝑖 is a vector of \ndistrict and state fixed ...,Does not contain dataset mention,1.0,1,NaN,NaN
3,"T he sample comprises 1,188 observations.",Does not contain dataset mention,1.0,1,NaN,NaN
4,"For \nparsimony, we drop the lagged dependent ...",Does not contain dataset mention,1.0,1,NaN,NaN
...,...,...,...,...,...,...
4891,"\nWoldeyohanes TR, Woldehaimanot TE, Kerie MW ...",Does not contain dataset mention,0.5,51,NaN,NaN
4892,World Bank Policy Research Working Paper No.,Contains dataset mention,0.5,51,NaN,NaN
4893,"For example, in Madagascar, in a context of cu...",Contains dataset mention,0.5,51,NaN,NaN
4894,"\n \nMoro, Andrea, Tomasz Piotr Wisniewski, an...",Does not contain dataset mention,0.5,51,NaN,NaN


In [ ]:
v1.query('DataBERT_class == "Does not contain dataset mention" & DataBERT_confidence == 0.99')['sentence'].to_list()

['\\n Budhram, Trevor.',
 '\\n Bloom, N., & Van Reenen, J.',
 'Euro Surveill.',
 'They must also keep records of daily income and ex-\\npenditure, sales and purchases of goods and services with names and Tax\\nIdentification Number (TIN), and relevant documents.',
 '\\n Chandra, A. and E. Thompson.',
 '1 5(1): e0227139.',
 'For example, Category A taxpayers must maintain detailed account-\\ning records following financial reporting standards.',
 '\\n \\nBloom, N., E. Brynjolfsson, L. Foster, R. Jarmin, M. Patnaik, I. Saporta -Eksten, J.',
 '\\n \\nBrookings Institution, 2021.',
 '\\nJ\\nEL classification : D22, D24, O12, O14, O18, R12 \\nA\\ncknowledgements: We thank the World Bank’s Research Support Budget for funding this paper.',
 'This result is plausible, as the period \\nconsidered in this paper immediately followed a major reorganization of the World Bank, one that might \\nhave temporarily detracted management attention from the supervision of analytical tasks.',
 'Economica, 8